## Imports

In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error

# Leer el CSV
data = pd.read_csv('../data/EstudioCrediticio_TrainP.csv')

## Preparación de Datos para Regresión

Se borran columnas Id y CreditoAprobado

In [51]:
# Eliminar la primera columna
data = data.iloc[:, 1:]

# Eliminar la penúltima columna
data = data.iloc[:, :-2].join(data.iloc[:, -1])

Obtener variables predictoras y dependiente

In [52]:
# Separar las características (X) y la variable a predecir (y)
X = data.iloc[:, :-1]  # Todas las columnas menos la última
y = data.iloc[:, -1]   # La última columna

Identificar variables categóricas y numéricas

In [53]:
# Identificar columnas numéricas y categóricas
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns

## Regresión Lineal

Creación de Pipeline para variables categóricas (SimpleImputer y OneHotEncoder)

In [54]:
# Pipeline para variables categóricas (Imputación + OneHotEncoder)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputar con el valor más frecuente
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

Creación de Pipeline para variables numéricas (SimpleImputer y RobustScaler)

In [55]:
# Pipeline para variables numéricas (Imputación + Escalado)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputar con la media
    ('scaler', RobustScaler())
])

Combinación de las dos transformaciones anteriores

In [56]:
# Combinación de transformaciones para columnas categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

Pipeline final con Preprocesamiento y Modelo

In [57]:
# Crear pipeline final con preprocesamiento y modelo
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

### Evaluación del método KFold para **validación cruzada**

In [58]:
# Definir la estrategia de validación cruzada con KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=73)

# Usar cross_val_score para evaluar el modelo utilizando KFold
results = cross_val_score(model, X, y, cv=kfold)

# Mostrar los resultados
print(f'Accuracy por fold: {results}')
print(f'Accuracy promedio: {results.mean()}')

Accuracy por fold: [0.76401947 0.72161892 0.76475537 0.75674145 0.76302607]
Accuracy promedio: 0.75403225592111


### Evaluación del modelo mediante train, test y métricas

División del conjunto de datos en train y test

In [59]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Entrenamiento** del modelo

In [60]:
# Entrenar el modelo
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  Index(['Edad', 'IngresoBrutoAnual', 'ScoreCrediticio', 'Experiencia',
       'CuantiaCredito', 'DuracionCredito', 'NumeroPersonasDependientes',
       'PagoMensualDeuda', 'TasaUtilizacionTarjetaCredito',
       'NumeroLineasCreditoAbiert...
       'TipoInteres', 'PagoMensualCredito', 'RatioTotalDeudaIngresos'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['SituacionLaboral', 'NivelEducativo', 'EstadoCivil', 'EstadoVivienda',
       'ObjetoCredito'],
      dtype='object'))])),
                ('regressor', LinearRegression())])

**Evaluación** del modelo

In [62]:
y_pred = model.predict(X_test)

# Calcular métricas para regresión
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R²: {r2}")

RMSE: 3.9395853507029934
MAE: 3.2006734374999994
R²: 0.7525255076333256


## Random Forest

Creación de Pipeline para variables categóricas (SimpleImputer y OneHotEncoder)

In [63]:
# Pipeline para variables categóricas (Imputación + OneHotEncoder)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputar con el valor más frecuente
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

Creación de Pipeline para variables numéricas (SimpleImputer y RobustScaler)

In [64]:
# Pipeline para variables numéricas (Imputación + Escalado)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputar con la media
    ('scaler', RobustScaler())
])

Combinación de las dos transformaciones anteriores

In [65]:
# Combinación de transformaciones para columnas categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

Pipeline final con Preprocesamiento y Modelo

In [66]:
# Crear pipeline final con preprocesamiento y modelo
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=73))
])

Grid Search de los mejores hiperparámetros utilizando Kfold para validación cruzada

In [85]:
# Definir el espacio de búsqueda para RandomizedSearchCV
param_dist = {
    'regressor__n_estimators': [100, 250],
    'regressor__max_depth': [10, 20]
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    model,                                    # Pipeline
    param_distributions=param_dist,           # Hiperparámetros (espacio de búsqueda)
    verbose=1,                                # Mostrar progreso en la consola
    n_jobs=-1,                                # Usar todos los núcleos disponibles
    random_state=73                           # Reproducibilidad
)

División en conjunto de entrenamiento y prueba

In [86]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Entrenamiento** del modelo

In [87]:
# Ajustar el modelo
random_search.fit(X_train, y_train)

C:\Users\guigr\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 4 candidates, totalling 20 fits


RandomizedSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer()),
                                                                                               ('scaler',
                                                                                                RobustScaler())]),
                                                                               Index(['Edad', 'IngresoBrutoAnual', 'ScoreCrediticio', 'Experiencia',
       'CuantiaCredito', 'DuracionCredito', 'NumeroPersonasDependientes',
       'PagoMensualDeuda', 'TasaUtilizacionTarjetaCredit...
                                                                                                SimpleImputer(strategy='most_frequent')),
                                                                                               ('onehot',
                                                                                                OneHotEncoder(handle_unknown='ignore'))]),
                                                                               Index(['SituacionLaboral', 'NivelEducativo', 'EstadoCivil', 'EstadoVivienda',
       'ObjetoCredito'],
      dtype='object'))])),
                                             ('regressor',
                                              RandomForestRegressor(random_state=73))]),
                   n_jobs=-1,
                   param_distributions={'regressor__max_depth': [10, 20],
                                        'regressor__n_estimators': [100, 250]},
                   random_state=73, verbose=1)

**Evaluación** del modelo

In [89]:
# Evaluar el modelo
train_score = random_search.score(X_train, y_train)
test_score = random_search.score(X_test, y_test)

print(f"R² del conjunto de entrenamiento: {train_score:.2f}")
print(f"R² del conjunto de prueba: {test_score:.2f}")

y_pred = random_search.predict(X_test)

# Calcular métricas para regresión
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R²: {r2}")

R² del conjunto de entrenamiento: 0.98
R² del conjunto de prueba: 0.87
RMSE: 2.8873541638346922
MAE: 1.7402659066522101
R²: 0.8670680026999077
